---
title: "Breast Cancer Classification with PyTorch"
format: html
author: "Nyjil Arackal"
date: "2025-01-30"
categories: [AI, Deep Learning, PyTorch, Healthcare]
image: "post2_cover.jpg"
---


### Introduction

Breast cancer is one of the most prevalent forms of cancer and **early detection** plays a crucial role in patient outcomes. In this project, we leverage deep learning with PyTorch to classify tumors as **benign (0)** or **malignant (1)** using the Breast Cancer Wisconsin dataset.

Through data preprocessing, neural network modeling, and performance evaluation, we create a model that achieves a **98.25%** accuracy in detecting malignancies.

### Dataset Overview

::::: columns
::: {.column width="50%"}
The Breast Cancer Wisconsin dataset is available in sklearn.datasets and contains **30** numerical features describing tumor properties.

The target variable consists of:\
   -   0  :   Benign tumors (non-cancerous)\
   -   1  :   Malignant tumors (cancerous)
:::

::: {.column width="40%"}
![](distribution.png)
:::
:::::

Let's start by loading the dataset:


In [ ]:
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X, y = dataset.data, dataset.target
X.shape, y.shape # 569 samples and 30 features

To visualize the data we convert it to Pandas DataFrame:


In [ ]:
import pandas as pd

dataf = pd.DataFrame(dataset.data, columns=dataset.feature_names)
dataf['target'] = dataset.target

Let's inspect a subset of the data:


In [ ]:
dataf[10:21]  # Showing samples 10 to 20
# in this segment only the samples 19 and 20 are malignant

### Data preprocessing

#### 1. Splitting the Dataset

Splitting into training (70%), validation (20%) and test (10%) sets:


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.33, random_state=42)

#### 2. Feature Scaling

To improve model training performance, we normalize the features using MinMaxScaler:


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

#### 3. Converting to PyTorch Tensors


In [ ]:
import torch

X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_valid, y_valid = torch.tensor(X_valid, dtype=torch.float32), torch.tensor(y_valid, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
X_train.shape, X_valid.shape, X_test.shape # sets: 398 training, 114 validation, 57 test

### Creating an Accuracy Function


In [ ]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item() # calculates the total number of correct predictions
  acc = (correct / len(y_pred)) * 100
  return acc

### Building the Neural Network

Creating the Model Class:


In [ ]:
from torch import nn
torch.manual_seed(42)

IN_FEATURES = X.shape[1] # 30
OUT_FEATURES = 9 # number of neurons in the hidden layers
OUTPUT = 1 # benign or malignant

class BreastCancerModel(nn.Module):
   def __init__(self):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
          nn.Linear(IN_FEATURES, OUT_FEATURES),
          nn.ReLU(),
          nn.Linear(OUT_FEATURES, OUT_FEATURES),
          nn.ReLU(),
          nn.Linear(OUT_FEATURES, OUTPUT)
          )

   def forward(self, x):
        return self.linear_layer_stack(x)

Setting up the Model:


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" # device agnostic code
BC_Model_V0 = BreastCancerModel().to(device) # initializing the model

### Loss Function and Optimizer:


In [ ]:
torch.manual_seed(42)

loss_fn = nn.BCEWithLogitsLoss() # Sigmoid built-in (better than: .BCELoss + Sigmoid)
optimizer = torch.optim.Adam(params = BC_Model_V0.parameters(),
                            lr = 0.001)

Moving data to GPU (if available):


In [ ]:
# data to GPU for faster computation
X_train, y_train = X_train.to(device), y_train.to(device)
X_valid, y_valid = X_valid.to(device), y_valid.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

### Training


In [ ]:
torch.manual_seed(42)
epochs = 500

# creating lists to track results
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

for epoch in range(epochs + 1):
    BC_Model_V0.train()
    y_logits = BC_Model_V0(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))

    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)
    
    train_losses.append(loss.item())
    train_accuracies.append(acc) 
    
    optimizer.zero_grad() # resetting gradients   
    loss.backward() # calculating gradients of loss_fn
    optimizer.step() # updating parameters
    
    BC_Model_V0.eval()
    with torch.inference_mode():
      
      valid_logits = BC_Model_V0(X_valid).squeeze()
      valid_pred = torch.round(torch.sigmoid(valid_logits))
      
      valid_loss = loss_fn(valid_logits, y_valid)
      valid_acc = accuracy_fn(y_true=y_valid,
                             y_pred=valid_pred)
      
      valid_losses.append(valid_loss.item())
      valid_accuracies.append(valid_acc)
      
    if (epoch % 50 == 0 or epoch == 500):
      print(f"Epoch: {epoch:3n} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Validation Loss: {valid_loss:.5f}, Validation Accuracy: {valid_acc:.2f}%")

Let's plot the Accuracy and the Loss functions.

The Accuracy:


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 6))
plt.plot(train_accuracies, label="Training Accuracy", color='blue')
plt.plot(valid_accuracies, label="Validation Accuracy", color='orange')
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.title("Model Accuracy: N_9 + lr_0.001")
plt.legend()
plt.show()

The Loss:


In [ ]:
plt.figure(figsize=(9, 6))

plt.plot(train_losses, label="Training Loss", color='green')
plt.plot(valid_losses, label="Validation Loss", color='red')

plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model Loss: N_9 + lr_0.001")
plt.legend()
plt.show()

### Evaluating the Model

Let's use the final 10% of the samples to see how the model performs on unseen data:


In [ ]:
with torch.inference_mode():
  BC_Model_V0.eval()
  test_logits = BC_Model_V0(X_test).squeeze()
  test_pred = torch.round(torch.sigmoid(test_logits))

  test_acc = accuracy_fn(y_true = y_test,
                         y_pred = test_pred)
print(f"Test accuracy: {test_acc:.5f}%")

### Hyperparameter Tuning and Architecture Adjustments

To optimize the model’s performance, I experimented with different hyperparameters values:

-   **Learning Rate** = **0.001** + **Neurons** = **8**\
    Training takes longer to reach the same accuracy (98.25%) and between 100 and 300 epochs it shows unstable behavior, a significant drop in accuracy occurs. This suggests that the model may experience optimization challenges - due to saddle point, poor local minima, momentum issues - before recovering.

::::: columns
::: {.column width="50%"}
![](Acc_N8_lr-0.001.png)
:::

::: {.column width="50%"}
![](Loss_N8_lr-0.001.png)
:::
:::::

-   **Learning Rate** = **0.01** + **Neurons** = **9**\
    The model converges faster (due to a higher learning rate - 0.01), but around epoch 250, overfitting occurs, as validation accuracy stagnates ( and then proceedes to decline) while training accuracy continues increasing. This suggests that the model has learned the training data too well - including the noise - and generalizes worse on unseen data.

::::: columns
::: {.column width="50%"}
![](Acc_N9_lr-0.01.png)
:::

::: {.column width="50%"}
![](Loss_N9_lr-0.01.png)
:::
:::::

### Conclusion

This post highlights the applications of machine learning in medicine, demonstrating how deep learning can assist in early **breast cancer detection**. While this model is relatively simple, it serves as a great example of **binary classification**, where the goal is to distinguish between two categories: benign and malignant tumors.

Even though the model achieves a high accuracy of **98.25%**, there are several ways to improve, such as:\
- **Hyperparameter tuning**: optimizing learning rates, batch sizes (if using a larger dataset) and architectures for better performance.\
- Using more complex models: implementing Convolutional Neural Networks (**CNNs**) for image-based cancer detection.\
- **Expanding the dataset**: including more diverse patient demographics to improve real-world generalization.

As machine learning continues to evolve, models like this can contribute to early disease detection, improving patient outcomes and supporting medical professionals in their decision-making.

### Links

-   [PyTorch Official Documentation](https://pytorch.org/docs/stable/index.html)
-   [Breast Cancer Wisconsin Dataset](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data/data)
-   [Learn PyTorch by Daniel Bourke](https://www.learnpytorch.io)

## License

[Cover image by Kjpargeter](https://www.freepik.com/author/kjpargeter)

::: {.callout-tip collapse="true" icon="file-code"}
### MIT License

MIT License\
Copyright (c) 2021 Daniel Bourke

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
:::